In [ ]:
%load_ext autoreload
%autoreload 2

import json
import openai
import os
import pandas as pd

from langchain.text_splitter import TokenTextSplitter
from pypdf import PdfReader

from functions import gpt

In [ ]:
product_name = 'mel_swan'
project_name = 'diygenomics'

data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, 'eric-client-projects', product_name, project_name, 'experiment-a', *args)

model = 'gpt-4' # 'gpt-3.5-turbo' # 'gpt-4'

# openai.api_key = os.getenv("DIYGENOMICS_OPENAI_API_KEY")
current_file_name = 's41467-022-33394-7.pdf' # '1 1999 Maldacena Large N Limit.pdf'

In [ ]:
system_prompt = """You are an academic researcher in information systems biology investigating possible links between DNA, 
RNA expression, and blood biomarkers for disease, especially aging and neurodegenerative pathologies 
(Alzheimer’s disease, Parkinson’s disease). 
- Please extract any gene that codes mentioned along with any relevant identifiers. 
- Please format your response in JSON.
- If you do not know the answer return {}
"""

In [ ]:
json_errors = {}
general_errors = {}

with open(file_path(current_file_name), 'rb') as pdf_file:
    reader = PdfReader(pdf_file)
    
    filename = os.path.splitext(os.path.basename(pdf_file.name))[0].replace(' ', '_')
    if not os.path.exists(file_path(filename)):
        os.mkdir(file_path(filename))
    
    metadata = {key[1:] if '/' in key else key: value for key, value in reader.metadata.items()}
    with open(file_path(filename, 'metadata.json'), 'w') as file:
        json.dump(metadata, file)


    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]

        full_text = page.extract_text()

        with open(file_path(filename, f'page_{page_num}.txt'), 'w') as file:
            file.write(full_text)
            
        text_splitter = TokenTextSplitter(chunk_size=6000, chunk_overlap=10)
        texts = text_splitter.split_text(full_text)
        
        for index, text in enumerate(texts):
            with open(file_path(filename, f'page_{page_num}_chunk_{index}.txt'), 'w') as file:
                file.write(text)
                
            possible_response = gpt.chat_create(system_prompt, text, model, output_json=True)
            
            if 'json_error' in possible_response:
                json_errors[row.name] = response
            elif 'general_error' in possible_response:
                general_errors[row.name] = response
            else:
                with open(file_path(filename, 'codes', f'{page_num}_chunk_{index}.json'), 'w') as file:
                    json.dump(possible_response, file)

In [ ]:
json_errors

In [ ]:
general_errors

In [ ]:
text = """
A meta-analysis of data from over 2000 donors identified over 300 cortical DMPs associated with tau pathology
We combined our BDR tau pathology EWAS results with the summary statistics from a recent analysis of tau pathology performed by our group13, performing a cross-cortex inverse variance weighted (IVW) meta-analysis of the Braak NFT stage including data for 403,763 DNA methylation sites from 2013 donors derived from seven independent cohorts (the 6 cohorts included in the Smith et al. meta-analysis13 in addition to the BDR samples described here (see Methods and Supplementary Data 7). In total, we identified 334 cortical DMPs (Bonferroni P < 1.24E–07) annotated to 171 genes (Fig. 3 and Supple- mentary Data 8). The full meta-analysis results for all probes tested are presented in Supplementary Data 9. Of note 140 (42% of the total) of these DMPs represented novel associations not previously identified in the previously published meta-analysis, reflecting the elevated power achieved by including the additional data from BDR donors. The top- ranked DMP, which was characterized by increasing DNA methylation with increased tau pathology (cg07061298: P = 8.06E–18, effect size =
0.32%, Fig. 3a) is annotated to HOXA3; of note, previous studies have strongly implicated differential DNA methylation across the HOXA region as being associated with AD pathology13,46,47, and we found that 17 (5%) of the 334 meta-analysis DMPs are annotated to this genomic region (Supplementary Fig. S16). We also confirmed other previous AD EWAS associations, including a site annotated to ANK1 (cg05066959; P = 1.16E–13, effect size = 0.41%) that has been robustly associated with AD pathology in previous EWAS studies of AD11,15,16 and was char- acterized by elevated DNA methylation with increased tau pathology (Fig. 3b). Interestingly, several of the identified DMPs are annotated to genes that been also been implicated in GWAS analyses of AD pathology, including cg06784824 (P = 1.71E–11, effect size = 0.21%, Fig. 3c) annotated to SPI1, a gene hypothesized to regulate AD- associated genes in primary human microglia7,48. We performed gene ontology (GO) pathway analysis of the 171 genes annotated to the significant DMPs in the cross-cortex meta-analysis using methylGSA (see Methods) identifying significant enrichment of multiple pathways including pathways related to immune and inflammatory processes (see Supplementary Data 10 and Supplementary Fig. S17). Mounting evidence suggests the immune system plays a role in the etiology of AD and other dementias49; both local and peripheral inflammation is triggered by the degeneration of tissues (e.g., damaged neurons and neurites) and the deposition and highly insoluble proteins such as Aβ and NFTs49. Of particular interest was an enrichment of DMPs associated with genes involved in metalloproteinase activity pathways (e.g., “metalloendopeptidase activity” [GO: 0004222, P = 5.09E–08]); these proteins are important in neuroinflammation and have been strongly linked to neurodegenerative disease50. Other GO categories enriched amongst genes annotated to DMPs associated with tau pathology include pathways implicated in AD including several related to mitochondrial function (e.g., “mitochondrial transport” [GO: 0006839, P = 5.09E–08]) and “unfolded protein binding” (GO: 0051082, P = 5.09E–08). We subsequently repeated the meta-analysis focusing only on DLPFC samples from 1545 individuals from four independent cohorts (the 3 DLPFC cohorts included in the Smith et al. meta-analysis13 in addition to the BDR DLPFC samples described here (see Methods and Supplementary Data 7), identifying 300 sig- nificant DMPs annotated to 161 genes (Supplementary Fig. S18 and
Supplementary Data 11). The full meta-analysis results for all probes tested are presented in Supplementary Data 12. There was consider- able overlap between the results from both meta-analyses with 215 DMPs being significant in both, and the direction of effect being 100% concordant between the cross-cortex DMPs (P = 2.86E–101) and DLPFC DMPs (P = 4.91E–91) (Supplementary Fig. S19).
An analysis of purified nuclei populations shows that the majority of DMPs identified in bulk cortex tissue reflect DNA methylation differences occurring in non-neuronal cells, with dramatically increased effect sizes observed in the NeuN–/ SOX10– immunolabeled nuclei population
Although we attempted to control for potential heterogeneity in the proportion of different cell-types in our analysis of bulk cortex DNA methylation by using novel reference panels generated on NeuN+ (neuron-enriched), SOX10+ (oligodendrocyte-enriched), and NeuN–/ SOX10– (microglia- and astrocyte-enriched) nuclei populations, our EWAS approach could not identify AD-associated differences occurring within specific cell populations. We therefore used our FANS protocol (see Methods) to profile DNA methylation in purified NeuN+, SOX10+, and NeuN–/SOX10– nuclei populations - in addition to a “total” nuclei population reflecting the cellular makeup of bulk cortex—from DLPFC tissue from a subset of “low” pathology (Braak score ≤II, n = 15) and “high” pathology (Braak score ≥V, n = 13) donors (Supplementary Data 13). We also co-stained nuclei with the microglial marker IRF8, highlighting complete overlap with the NeuN–/SOX10– population. Of note, a large proportion of NeuN–/SOX10– nuclei were IRF8+ (mean = 42.23%) indicating a relatively strong enrichment of microglia amongst this double-negative population. Of the DMPs identified in the DLPFC tau pathology EWAS meta-analysis, we obtained data for 327 sites in the purified nuclei populations (n = 327 DMPs). First, we looked at between- group effect sizes in the “total” nuclei population finding highly con- sistent DNA methylation differences to those seen in the large DLPFC meta-analysis despite the small number of samples, confirming the validity of our EWAS results (sign-test P = 7.24E–46, 87% concordant direction of effect). We then examined high vs low Braak score differ- ences in DNA methylation at the 327 DLPFC DMPs finding a striking difference in the consistency and magnitude of effect sizes across each of the nuclei populations (Fig. 4). Although 67 DMPs (20%) had con- sistent directions of effects across all nuclei populations (Supplemen- tary Data 14), the NeuN–/SOX10– population showed the most consistent between-group differences in DNA methylation (sign-test P = 1.2E–75, 96% concordant direction of effect) and was also char- acterized by a dramatic increase in effect sizes compared to those observed in bulk DLPFC (mean fold-change in effect size compared to bulk DLPFC = 10.72, Fig. 4). A similar pattern of differential DNA methylation was also observed in the SOX10+ (oligodendrocyte-enri- ched) population (sign-test P = 2.15E–10, 67% concordant direction of effect) again with an elevated effect sizes compared to bulk DLPFC, albeit to a lesser extent (mean fold-change in effect size compared to bulk DLPFC = 1.93, Fig. 4). These results suggest that the widespread cortical DNA methylation differences associated with AD neuropathol- ogy are primarily manifest in non-neuronal cell-types, although there is evidence for pathology-associated differences in cortical DNA methy- lation being specifically driven by variation in neuronal cell-types types for a subset (n = 27 (8.3%)) of tested DMPs (Supplementary Data 14)."""

In [ ]:
# copied from word
text = """
We combined our BDR tau pathology EWAS results with the summary statistics from a recent analysis of tau pathology performed by our group  ^13, performing a cross-cortex inverse variance weighted (IVW) meta-analysis of the Braak NFT stage including data for 403,763 DNA methylation sites from 2013 donors derived from seven independent cohorts (the 6 cohorts included in the Smith et al. meta-analysis  ^13 in addition to the BDR samples described here (see Methods and Supplementary Data 7). In total, we identified 334 cortical DMPs (Bonferroni P<1.24E-07 ) annotated to 171 genes (Fig. 3 and Supplementary Data 8). The full meta-analysis results for all probes tested are presented in Supplementary Data 9. Of note 140 (42% of the total) of these DMPs represented novel associations not previously identified in the previously published meta-analysis, reflecting the elevated power achieved by including the additional data from BDR donors. The topranked DMP, which was characterized by increasing DNA methylation with increased tau pathology (cg07061298: P=8.06E-18, effect size = 0.32%, Fig. 3a) is annotated to HOXA3; of note, previous studies have strongly implicated differential DNA methylation across the HOXA region as being associated with AD pathology  ^13,46,47, and we found that 17 (5%) of the 334 meta-analysis DMPs are annotated to this genomic region (Supplementary Fig. S16). We also confirmed other previous AD EWAS associations, including a site annotated to ANK1 (cg05066959; P=1.16E-13, effect size =0.41%) that has been robustly associated with AD pathology in previous EWAS studies of AD^11,15,16 and was characterized by elevated DNA methylation with increased tau pathology (Fig. 3b). Interestingly, several of the identified DMPs are annotated to genes that been also been implicated in GWAS analyses of AD pathology, including cg06784824 (P=1.71E-11, effect size =0.21%, Fig. 3c) annotated to SPI1, a gene hypothesized to regulate ADassociated genes in primary human microglia  ^7,48. We performed gene ontology (GO) pathway analysis of the 171 genes annotated to the significant DMPs in the cross-cortex meta-analysis using methylGSA (see Methods) identifying significant enrichment of multiple pathways including pathways related to immune and inflammatory processes (see Supplementary Data 10 and Supplementary Fig. S17). Mounting evidence suggests the immune system plays a role in the etiology of AD and other dementias  ^49; both local and peripheral inflammation is triggered by the degeneration of tissues (e.g., damaged neurons and neurites) and the deposition and highly insoluble proteins such as Aβ and NFTs^49. Of particular interest was an enrichment of DMPs associated with genes involved in metalloproteinase activity pathways (e.g., "metalloendopeptidase activity" [GO: 0004222, P=5.09E-08] ); these proteins are important in neuroinflammation and have been strongly linked to neurodegenerative disease  ^50. Other GO categories enriched amongst genes annotated to DMPs associated with tau pathology include pathways implicated in AD including several related to mitochondrial function (e.g., "mitochondrial transport" [GO: 0006839, P=5.09E-08] ) and "unfolded protein binding" (GO: 0051082,P=5.09E-08). We subsequently repeated the meta-analysis focusing only on DLPFC samples from 1545 individuals from four independent cohorts (the 3 DLPFC cohorts included in the Smith et al. meta-analysis  ^13 in addition to the BDR DLPFC samples described here (see Methods and Supplementary Data 7), identifying 300 significant DMPs annotated to 161 genes (Supplementary Fig. S18 and Supplementary Data 11). The full meta-analysis results for all probes tested are presented in Supplementary Data 12. There was considerable overlap between the results from both meta-analyses with 215 DMPs being significant in both, and the direction of effect being 100% concordant between the cross-cortex DMPs (P=2.86E-101) and DLPFC DMPs (P=4.91E-91) (Supplementary Fig. S19).
An analysis of purified nuclei populations shows that the majority of DMPs identified in bulk cortex tissue reflect DNA methylation differences occurring in non-neuronal cells, with dramatically increased effect sizes observed in the NeuN-/ SOX10- immunolabeled nuclei population
Although we attempted to control for potential heterogeneity in the proportion of different cell-types in our analysis of bulk cortex DNA methylation by using novel reference panels generated on NeuN+ (neuron-enriched), SOX10+ (oligodendrocyte-enriched), and NeuN-/ SOX10- (microglia- and astrocyte-enriched) nuclei populations, our EWAS approach could not identify AD-associated differences occurring within specific cell populations. We therefore used our FANS protocol (see Methods) to profile DNA methylation in purified NeuN+, SOX10+, and NeuN-/SOX10- nuclei populations - in addition to a "total" nuclei population reflecting the cellular makeup of bulk cortex-from DLPFC tissue from a subset of "low" pathology (Braak score ≤ II, n=15 ) and "high" pathology (Braak score ≥V,n=13 ) donors (Supplementary Data 13). We also co-stained nuclei with the microglial marker IRF8, highlighting complete overlap with the NeuN-/SOX10 - population. Of note, a large proportion of NeuN-/SOX10- nuclei were IRF8+ (mean = 42.23%) indicating a relatively strong enrichment of microglia amongst this double-negative population. Of the DMPs identified in the DLPFC tau pathology EWAS meta-analysis, we obtained data for 327 sites in the purified nuclei populations ( n=327 DMPs). First, we looked at betweengroup effect sizes in the "total" nuclei population finding highly consistent DNA methylation differences to those seen in the large DLPFC meta-analysis despite the small number of samples, confirming the validity of our EWAS results (sign-test P=7.24E-46,87% concordant direction of effect). We then examined high vs low Braak score differences in DNA methylation at the 327 DLPFC DMPs finding a striking difference in the consistency and magnitude of effect sizes across each of the nuclei populations (Fig. 4). Although 67 DMPs (20%) had consistent directions of effects across all nuclei populations (Supplementary Data 14), the NeuN-/SOX10- population showed the most consistent between-group differences in DNA methylation (sign-test P=1.2E-75,96% concordant direction of effect) and was also characterized by a dramatic increase in effect sizes compared to those observed in bulk DLPFC (mean fold-change in effect size compared to bulk DLPFC =10.72, Fig. 4). A similar pattern of differential DNA methylation was also observed in the SOX10+ (oligodendrocyte-enriched) population (sign-test P=2.15E-10,67% concordant direction of effect) again with an elevated effect sizes compared to bulk DLPFC, albeit to a lesser extent (mean fold-change in effect size compared to bulk DLPFC =1.93, Fig. 4). These results suggest that the widespread cortical DNA methylation differences associated with AD neuropathology are primarily manifest in non-neuronal cell-types, although there is evidence for pathology-associated differences in cortical DNA methylation being specifically driven by variation in neuronal cell-types types for a subset ( n=27(8.3%) ) of tested DMPs (Supplementary Data 14).
"""

In [ ]:
possible_response = gpt.chat_create(system_prompt, text, model, output_json=True)

In [ ]:
possible_response

In [ ]:
# copied from tex
text = """
We combined our BDR tau pathology EWAS results with the summary statistics from a recent analysis of tau pathology performed by our group \(^{13}\), performing a cross-cortex inverse variance weighted (IVW) meta-analysis of the Braak NFT stage including data for 403,763 DNA methylation sites from 2013 donors derived from seven independent cohorts (the 6 cohorts included in the Smith et al. meta-analysis \({ }^{13}\) in addition to the BDR samples described here (see Methods and Supplementary Data 7). In total, we identified 334 cortical DMPs (Bonferroni \(P<1.24 \mathrm{E}-07\) ) annotated to 171 genes (Fig. 3 and Supplementary Data 8). The full meta-analysis results for all probes tested are presented in Supplementary Data 9. Of note 140 (42\% of the total) of these DMPs represented novel associations not previously identified in the previously published meta-analysis, reflecting the elevated power achieved by including the additional data from BDR donors. The topranked DMP, which was characterized by increasing DNA methylation with increased tau pathology (cg07061298: \(P=8.06 \mathrm{E}-18\), effect size \(=\) \(0.32 \%\), Fig. 3a) is annotated to \(H O X A 3\); of note, previous studies have strongly implicated differential DNA methylation across the HOXA region as being associated with \(\mathrm{AD}\) pathology \({ }^{13,46,47}\), and we found that 17 (5\%) of the 334 meta-analysis DMPs are annotated to this genomic region (Supplementary Fig. S16). We also confirmed other previous AD EWAS associations, including a site annotated to ANK1 (cg05066959; \(P=1.16 \mathrm{E}-13\), effect size \(=0.41 \%)\) that has been robustly associated with \(\mathrm{AD}\) pathology in previous EWAS studies of \(\mathrm{AD}^{11,15,16}\) and was characterized by elevated DNA methylation with increased tau pathology (Fig. 3b). Interestingly, several of the identified DMPs are annotated to genes that been also been implicated in GWAS analyses of AD pathology, including cg06784824 \((P=1.71 \mathrm{E}-11\), effect size \(=0.21 \%\), Fig. 3c) annotated to SPI1, a gene hypothesized to regulate ADassociated genes in primary human microglia \({ }^{7,48}\). We performed gene ontology (GO) pathway analysis of the 171 genes annotated to the significant DMPs in the cross-cortex meta-analysis using methylGSA (see Methods) identifying significant enrichment of multiple pathways including pathways related to immune and inflammatory processes (see Supplementary Data 10 and Supplementary Fig. S17). Mounting evidence suggests the immune system plays a role in the etiology of \(A D\) and other dementias \({ }^{49}\); both local and peripheral inflammation is triggered by the degeneration of tissues (e.g., damaged neurons and neurites) and the deposition and highly insoluble proteins such as \(\mathrm{A} \beta\) and \(\mathrm{NFTs}^{49}\). Of particular interest was an enrichment of DMPs associated with genes involved in metalloproteinase activity pathways (e.g., "metalloendopeptidase activity" [GO: 0004222, \(P=5.09 \mathrm{E}-08]\) ); these proteins are important in neuroinflammation and have been strongly linked to neurodegenerative disease \({ }^{50}\). Other GO categories enriched amongst genes annotated to DMPs associated with tau pathology include pathways implicated in \(\mathrm{AD}\) including several related to mitochondrial function (e.g., "mitochondrial transport" [GO: 0006839, \(P=5.09 \mathrm{E}-08]\) ) and "unfolded protein binding" (GO: \(0051082, P=5.09 \mathrm{E}-08)\). We subsequently repeated the meta-analysis focusing only on DLPFC samples from 1545 individuals from four independent cohorts (the 3 DLPFC cohorts included in the Smith et al. meta-analysis \({ }^{13}\) in addition to the BDR DLPFC samples described here (see Methods and Supplementary Data 7), identifying 300 significant DMPs annotated to 161 genes (Supplementary Fig. S18 and Supplementary Data 11). The full meta-analysis results for all probes tested are presented in Supplementary Data 12. There was considerable overlap between the results from both meta-analyses with 215 DMPs being significant in both, and the direction of effect being \(100 \%\) concordant between the cross-cortex DMPs \((P=2.86 \mathrm{E}-101)\) and DLPFC DMPs \((P=4.91 \mathrm{E}-91)\) (Supplementary Fig. S19).

An analysis of purified nuclei populations shows that the majority of DMPs identified in bulk cortex tissue reflect DNA methylation differences occurring in non-neuronal cells, with dramatically increased effect sizes observed in the NeuN-/ SOX10- immunolabeled nuclei population

Although we attempted to control for potential heterogeneity in the proportion of different cell-types in our analysis of bulk cortex DNA methylation by using novel reference panels generated on \(\mathrm{NeuN}+\) (neuron-enriched), SOX10+ (oligodendrocyte-enriched), and \(\mathrm{NeuN}-/\) SOX10- (microglia- and astrocyte-enriched) nuclei populations, our EWAS approach could not identify AD-associated differences occurring within specific cell populations. We therefore used our FANS protocol (see Methods) to profile DNA methylation in purified NeuN+, SOX10+, and NeuN-/SOX10- nuclei populations - in addition to a "total" nuclei population reflecting the cellular makeup of bulk cortex-from DLPFC tissue from a subset of "low" pathology (Braak score \(\leq\) II, \(n=15\) ) and "high" pathology (Braak score \(\geq \mathrm{V}, n=13\) ) donors (Supplementary Data 13). We also co-stained nuclei with the microglial marker IRF8, highlighting complete overlap with the \(\mathrm{NeuN}-/ \mathrm{SOX10}\) - population. Of note, a large proportion of NeuN-/SOX10- nuclei were IRF8+ (mean = 42.23\%) indicating a relatively strong enrichment of microglia amongst this double-negative population. Of the DMPs identified in the DLPFC tau pathology EWAS meta-analysis, we obtained data for 327 sites in the purified nuclei populations ( \(n=327\) DMPs). First, we looked at betweengroup effect sizes in the "total" nuclei population finding highly consistent DNA methylation differences to those seen in the large DLPFC meta-analysis despite the small number of samples, confirming the validity of our EWAS results (sign-test \(P=7.24 \mathrm{E}-46,87 \%\) concordant direction of effect). We then examined high vs low Braak score differences in DNA methylation at the 327 DLPFC DMPs finding a striking difference in the consistency and magnitude of effect sizes across each of the nuclei populations (Fig. 4). Although 67 DMPs (20\%) had consistent directions of effects across all nuclei populations (Supplementary Data 14), the NeuN-/SOX10- population showed the most consistent between-group differences in DNA methylation (sign-test \(P=1.2 \mathrm{E}-75,96 \%\) concordant direction of effect) and was also characterized by a dramatic increase in effect sizes compared to those observed in bulk DLPFC (mean fold-change in effect size compared to bulk DLPFC \(=10.72\), Fig. 4). A similar pattern of differential DNA methylation was also observed in the \(\mathrm{SOX} 10+\) (oligodendrocyte-enriched) population (sign-test \(P=2.15 \mathrm{E}-10,67 \%\) concordant direction of effect) again with an elevated effect sizes compared to bulk DLPFC, albeit to a lesser extent (mean fold-change in effect size compared to bulk DLPFC \(=1.93\), Fig. 4). These results suggest that the widespread cortical DNA methylation differences associated with AD neuropathology are primarily manifest in non-neuronal cell-types, although there is evidence for pathology-associated differences in cortical DNA methylation being specifically driven by variation in neuronal cell-types types for a subset ( \(n=27(8.3 \%)\) ) of tested DMPs (Supplementary Data 14).
"""

In [ ]:
possible_response = gpt.chat_create(system_prompt, text, model, output_json=True)

In [ ]:
possible_response